# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
kgrid = [1, 1, 1]
Ecut = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.

In [4]:
struct MyMixing
    α  # Damping parameter
end
MyMixing() = MyMixing(0.7)

function DFTK.mix(mixing::MyMixing, basis, ρin::RealFourierArray, ρout::RealFourierArray; n_iter, kwargs...)
    if n_iter <= 2
        # Just do simple mixing
        ρin + mixing.α * (ρout - ρin)
    else
        # Use the KerkerMixing from DFTK
        DFTK.mix(KerkerMixing(α=mixing.α), basis, ρin, ρout; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

Iter   Energy             ρout-ρin
----   ------             --------
  1    -7.081453498029    4.156439E-01
  2    -7.228454013481    2.238251E-01
  3    -7.246573815228    9.127266E-02
  4    -7.248418907195    4.961564E-02
  5    -7.248978587342    2.735252E-02
  6    -7.249145081389    1.535809E-02
  7    -7.249194792612    8.791868E-03
  8    -7.249209893514    5.125975E-03
  9    -7.249214601466    3.037390E-03
 10    -7.249216116330    1.824699E-03
 11    -7.249216620940    1.108799E-03
 12    -7.249216795079    6.802178E-04
 13    -7.249216857255    4.206424E-04
 14    -7.249216880157    2.618984E-04
 15    -7.249216888826    1.640234E-04


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.